<a href="https://colab.research.google.com/github/ctvidal/Text-Mining/blob/master/2_Aquisi%C3%A7%C3%A3o_de_Dados_da_Web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aquisição de Dados

O primeiro processo da Mineração de Texto, é garantir que temos os dados necessários para análise. A mineração de texto é muito dinâmica e provavelmente temos que montar um **fluxo contínuo** de aquisição dos dados necessário. Neste notebook vamos exemplificar como recuperar dados de uma página HTML e estruturá-los.

### Importando bibliotecas a serem usadas

In [2]:
install.packages('rvest')
install.packages('dplyr')
install.packages('stringr')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


In [0]:
suppressMessages(suppressWarnings(library(rvest)))
suppressMessages(suppressWarnings(library(dplyr)))
suppressMessages(suppressWarnings(library(stringr)))

### Download dos dados

### 1) Escolha uma URL para fazer download dos dados

In [0]:
#URL contendo as listagem dos produtos do Supermercado Guanabara
url <- "https://www.supermercadosguanabara.com.br/produtos"

In [0]:
webpage <- read_html(url)


###  2) Escreva quais informações você gostaria de estruturar:

As promoções do Supermercado Guanabara se encontram divididas em grupos de alimentos/produtos e em cada grupo é fornecido a listagem paginada dos nomes das mercadorias e seus respectivos preços.

O objetivo do código é extrair toda a listagem desses produtos, com os preços além de informações de validade da promoção dos itens.

Este procedimento pode ser feito para vários supermercados de forma a obtermos ao final uma base de dados que posibilite alimentar sistemas de busca de melhores preços.


### 3) Escreva o código  para recuperar essas informações a partir da página especificada

In [13]:
#Deteminando as páginas dos grupos de produtos para varredura
paginas <- html_nodes(webpage,'.item') %>% html_attr("href") 
paginas <- paginas[!is.na(paginas)]

paginas

[1] "/produtos/42"  "/produtos/82"  "/produtos/32"  "/produtos/152"
 [5] "/produtos/182" "/produtos/2"   "/produtos/92"  "/produtos/52" 
 [9] "/produtos/62"  "/produtos/112" "/produtos/102" "/produtos/22" 
[13] "/produtos/12"  "/produtos/132" "/produtos/72"

In [37]:
#Varrendo cada página e extraindo as informações dos produtos

data <- vector("list", length = 1)
k=1

for(i in 1:length(paginas)) {
  
  url <- paste0('https://www.supermercadosguanabara.com.br',paginas[i])
  webpage <- read_html(url)
  
  rank_data_html <- html_nodes(webpage,'.validate')
  
  rank_data <- html_text(rank_data_html,trim = TRUE) %>% str_split(pattern = ' ') %>% unlist()
  
  promo_fim <- as.Date(rank_data[2],"%d/%m/%Y")
  
  nome_prod <- html_nodes(webpage,'.name') %>% html_text(trim = TRUE) %>% unlist() 
  nome_prod <- nome_prod[length(paginas)+1:length(nome_prod)]
  nome_prod <- nome_prod[!is.na(nome_prod)]
  
  preco_prod <- html_nodes(webpage,'.price') %>% html_text(trim = TRUE) %>% 
    str_replace_all(pattern = " ",replacement = "") %>%
    str_replace_all(pattern = "cada",replacement = "") %>% 
    str_replace_all(pattern = "[R$]",replacement = "") %>% unlist
  
  if (length(nome_prod)>0) {
    preco_prod <- as.numeric(str_replace(preco_prod,",","."))
    data <- rbind(data, data.frame("Validade Promoção"=promo_fim,"Produto"=nome_prod,"Preço"=preco_prod))
  }
}

summary(data)

 Validade.Promoção                                                 Produto   
 Min.   :2019-06-15   Hastes Flexíveis Cotonetes Johnson e Johnson 75un:  2  
 1st Qu.:2019-06-15   Alcatra Bovina a Vácuo (Peça) Kg                 :  1  
 Median :2019-06-15   Contra Filé Friboi PEÇA Kg                       :  1  
 Mean   :2019-06-15   Coraçao da Alcatra Maturatta Friboi PEÇA Kg      :  1  
 3rd Qu.:2019-06-15   Coxa de Frango Sadia Bandeja Kg                  :  1  
 Max.   :2019-06-15   Filé de Coxa e Sobrecoxa Sadia Bandeja Kg        :  1  
                      (Other)                                          :241  
     Preço       
 Min.   : 0.790  
 1st Qu.: 2.990  
 Median : 6.480  
 Mean   : 8.279  
 3rd Qu.: 9.990  
 Max.   :77.900  
                 

## Estruturando os dados

#### 4) Coloque seus dados em um dataframe e salve em CSV para recuperarmos mais tarde

In [41]:
base_final <- data.frame(data)

head(base_final)

Validade.Promoção,Produto,Preço
<date>,<fct>,<dbl>
2019-06-15,Coxa de Frango Sadia Bandeja Kg,7.98
2019-06-15,Filezinho de Peito de Frango Sadia Bandeja,13.98
2019-06-15,Filezinho de Peito de Frango Seara Band Kg,13.98
2019-06-15,Filé de Coxa e Sobrecoxa Sadia Bandeja Kg,9.98
2019-06-15,Frango a Passarinho Seara Pacote kg,7.95
2019-06-15,Linguiça Suína Alegra Kg,9.98


In [0]:
write.csv(base_final, "ProdutosxPreços_GUANABARA.csv")